In [38]:
import requests, json
import os
from dotenv import load_dotenv

#make an API call to Brightdata to beging the collection of image urls
load_dotenv()
BRIGHTDATA_API_KEY = os.getenv('brightdata') # Replace with your API key

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {BRIGHTDATA_API_KEY}"
}
data = [{"url":"https://www.memedroid.com/memes/top/day?page=1"}]
url = "https://api.brightdata.com/dca/trigger?collector=c_mdmeul7w21y46152n8&queue_next=1"

# Make the POST request
response = requests.post(url, headers=headers, json=data)

# Print response
print("Status code:", response.status_code)
#print("Response JSON:", response.json())
response = response.json()
collectionId = response['collection_id']
print(collectionId)


Status code: 200
Response JSON: {'collection_id': 'j_mdmfwgz02g8158vxs2', 'start_eta': '2025-07-28T01:39:05.802Z'}
j_mdmfwgz02g8158vxs2


In [40]:
#use the collectionId to get the Json with urls
url = f"https://api.brightdata.com/dca/dataset?id={collectionId}"
response = requests.get(url, headers=headers)
responseJson =response.json()





[{'memes': [{'title': 'tvPost', 'image_url': 'https://images3.memedroid.com/images/UPLOADED593/6886b571db6ae.jpeg', 'author': 'AdVictorium', 'posted_time': '1h', 'rating_percentage': '95%', 'vote_count': 59, 'tags': ['Liberalismisamentaldisorder', 'Women Are The Devil'], 'meme_url': 'https://www.memedroid.com/memes/detail/4558776/tvPost?refGallery=topDay&page=1', 'item_id': '4558776'}, {'title': 'It do be like that', 'image_url': 'https://images3.memedroid.com/images/UPLOADED358/6886111912688.jpeg', 'author': 'DennisvO', 'posted_time': '13h', 'rating_percentage': '94%', 'vote_count': 579, 'tags': [], 'meme_url': 'https://www.memedroid.com/memes/detail/4558651/It-do-be-like-that?refGallery=topDay&page=1', 'item_id': '4558651'}, {'title': 'Title', 'image_url': 'https://images7.memedroid.com/images/UPLOADED280/68860f05d88b0.jpeg', 'author': 'TBIGUY', 'posted_time': '13h', 'rating_percentage': '91%', 'vote_count': 658, 'tags': [], 'meme_url': 'https://www.memedroid.com/memes/detail/4558645

In [53]:
#download all the images from the scraping results

#directory for saving the files
destination_dir = "documents"

#meme list
memes = responseJson[0]['memes']

for meme in memes:
    url = meme['image_url']
    #get the image and save it locally
    if url is not None:
        filename = url.split("/")[-1]  # Get filename from URL
        destination_path = os.path.join(destination_dir, filename)
        # Download the file
        response = requests.get(url)
        # Save the file if request was successful
        if response.status_code == 200:
            with open(destination_path, "wb") as f:
                f.write(response.content)
            print(f"Downloaded to: {destination_path}")
        else:
            print(f"Failed to download. Status code: {response.status_code}")


Downloaded to: documents/6886b571db6ae.jpeg
Downloaded to: documents/6886111912688.jpeg
Downloaded to: documents/68860f05d88b0.jpeg
Downloaded to: documents/6886aac8477f2.jpeg
Downloaded to: documents/68864018dc3f1.jpeg
Downloaded to: documents/6886aa8648df6.jpeg
Downloaded to: documents/688669c5bc7f0.jpeg
Downloaded to: documents/68862d3df0fcd.jpeg
Downloaded to: documents/6886781a1cb64.jpeg
Downloaded to: documents/68867ee7cc11c.jpeg
Downloaded to: documents/6886a9e7c2fd2.jpeg
Downloaded to: documents/688669b78505b.jpeg
Downloaded to: documents/6885fe130390c.jpeg
Downloaded to: documents/6886a9b6d85e1.jpeg
Downloaded to: documents/68840ce56881a.jpeg
Downloaded to: documents/68869a5da356d.jpeg
Downloaded to: documents/68857a2124fd8.jpeg


In [54]:
##extract text from images and PDFs

import os
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import PyPDF2
import datetime

def extract_text_from_pdf(file_path):

    #Extract text from a PDF file.

    pdf_file_obj = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
    num_pages = len(pdf_reader.pages) 
    text = ''
    for page in range(num_pages):
        page_obj = pdf_reader.pages[page]
        text += page_obj.extract_text()
    pdf_file_obj.close()
    return text

def extract_text_from_images(file_path):
   
    #Extract text from image file.
    image = Image.open(file_path)
    text = pytesseract.image_to_string(image)
    return text

def extract_text_from_files(directory_path):
    
    #Extract text from PDF files and images in a directory.
    #this can be extended and modified asn more file types are added to the ingestion engine
    
    #will hold texts and associated metadata
    texts = []
    filetype = ""
    #counters are helpful in debugging
    img_count = 0
    pdf_count=0
    
    #loop through all the files in the directory. Determine filetype, and process accordingly
    for filename in os.listdir(directory_path):
        #create the full file path
        path = os.path.join(directory_path, filename)
        #file creation time (assuming linux) convert to a datetime object
        stat = os.stat(path)
        birth_time = stat.st_birthtime
        created_at = datetime.datetime.fromtimestamp(birth_time)
        txt = ""
        if filename.endswith(".pdf"):
            #pdf
            filetype="pdf"
            txt = extract_text_from_pdf(path)
            pdf_count+=1
            
        elif filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            filetype="image"
            txt = extract_text_from_images(path)
            img_count+=1

        #create an object with the text and the metadata, insert it into the texts arraat
        temp = {'type':filetype, 'filename':filename, 'created_at':created_at,'text':txt}
        texts.append(temp)
    #all texted parsed. Print the stats, and return the data
    print(f"parsed {img_count} image(s) and {pdf_count} pdf(s).")
    return texts




# Example usage
directory_path = '/Users/dougsillars/Github/lettria/documents'
extracted_text = []
extracted_text = extract_text_from_files(directory_path)
#for text in extracted_text:
#    print(text)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

parsed 21 image(s) and 0 pdf(s).


In [55]:
#create the embeddings
#summarize, sentiment and pull entities from the data


#if using a mac - this fixes an issue
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"  
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import spacy


# Load tools
nlp = spacy.load("en_core_web_sm")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)  # force CPU, no progress bar
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    device=-1  # Forces CPU
)

embedder = SentenceTransformer("all-MiniLM-L6-v2")

def enrich_chunk(text, source=None):
    if not text.strip():
        return None

    # NER
    doc = nlp(text)
    entities = list(set(ent.text for ent in doc.ents))

    # Summary (trim to 512 tokens max)
    try:
        token_count = len(text.split())
        max_length = 60
        if token_count < max_length // 2:
            # If text is very short, just return it as-is (or shorten max_length)
            summary = text
        else:
            summary_result = summarizer(text[:1024], max_length=max_length, min_length=20, do_sample=False)
            summary = summary_result[0]['summary_text']
    except Exception as e:
        summary = None
        print(f"[summary error] {e}")

    # Sentiment
    try:
        sentiment_result = sentiment_analyzer(text[:512])[0]
        sentiment_label = sentiment_result['label']
        sentiment_score = sentiment_result['score']
    except Exception as e:
        sentiment_label, sentiment_score = "ERROR", 0.0
        print(f"[sentiment error] {e}")

    # Embedding
    embedding = embedder.encode(text)

    return {
        "text": text,
        "summary": summary,
        "entities": entities,
        "sentiment": {
            "label": sentiment_label,
            "score": round(sentiment_score, 3),
        },
        "embedding": embedding.tolist(),
        "source": source
    }


Device set to use cpu
Device set to use cpu


In [56]:
example = "Walk softly, but carry a big stick - Teddy Roosevelt"
result = enrich_chunk(example)
import pprint
#pprint.pprint(result)

/Users/dougsillars/anaconda3/envs/bittensor8/lib/python3.12/site-packages/torch/nn/modules/module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [57]:
import pandas as pd
import re
# Assume 'extracted_text' is the output of the text extraction function
data = []

for text in extracted_text:
    # Split the text into individual words or tokens
    #this depends on the type of file.. images - all text in one
    #PDFs split by new lines
    filetype = text['type']
    if filetype == "image":
        #full text
        tokens = [text['text']]
    else:
        #pdf - split on every newline
        tokens = re.split(r'[\n|\ue002]', text['text'])
    
    # Remove empty tokens
    tokens = [t for t in tokens if t.strip()]

    for token in tokens:
        #create the embedding 
        #and 
        #get entities (named people)
        # get sentimatent analysis
        #set a summary
        enriched = enrich_chunk(token)
        if enriched is not None:
            #only add an embedding if everything is successfully extracted
            summary = enriched['summary']
            entities = enriched['entities']
            sentiment = enriched['sentiment']
            embedding = enriched['embedding']
            # Create a new array for each entry
            #keep all the metadata from extraction
            entry = {
                'filetype': filetype,
                'filename':text['filename'],
                'created_at': text['created_at'],
                'tokens': token,
                'summary':summary, 
                'entities':entities,
                'sentiment':sentiment,
                'embedding':embedding
            }
            data.append(entry)

# Create a data frame from the data list
df = pd.DataFrame(data)

In [58]:
print(df)


   filetype            filename                 created_at  \
0     image  6886111912688.jpeg 2025-07-27 21:48:52.193284   
1     image  6886a9b6d85e1.jpeg 2025-07-27 21:50:16.695512   
2     image  68860f05d88b0.jpeg 2025-07-27 21:50:15.114664   
3     image  6886781a1cb64.jpeg 2025-07-27 21:50:15.951250   
4     image              TR.jpg 2025-07-21 21:24:38.382369   
5     image       twocities.png 2025-07-21 20:40:41.575967   
6     image  6886a9e7c2fd2.jpeg 2025-07-27 21:50:16.205884   
7     image  68869a5da356d.jpeg 2025-07-27 21:50:17.015520   
8     image  68864018dc3f1.jpeg 2025-07-27 21:50:15.415774   
9     image  6885fe130390c.jpeg 2025-07-27 21:50:16.498120   
10    image  6886b571db6ae.jpeg 2025-07-27 21:50:14.749833   
11    image  6886aac8477f2.jpeg 2025-07-27 21:50:15.251529   
12    image  688669b78505b.jpeg 2025-07-27 21:50:16.351617   
13    image  68867ee7cc11c.jpeg 2025-07-27 21:50:16.074208   
14    image  6886aa8648df6.jpeg 2025-07-27 21:50:15.543983   
15    im

In [59]:
import numpy as np
import faiss


# Convert embeddings to NumPy array
embedding_matrix = np.array(df['embedding'].tolist()).astype('float32')
#Build a FAISS index
dimension = embedding_matrix.shape[1]  # number of features in each vector
index = faiss.IndexFlatL2(dimension)
index.add(embedding_matrix)

#Metadata as in memory database
metadata = df.to_dict(orient='records')

In [33]:

# Example: search for the 3 most similar items to the first vector
query_vector = embedding_matrix[0:1]  # shape (1, dim)
D, I = index.search(query_vector, k=3)

# Display results
for idx in I[0]:
    print(metadata[idx])  # full metadata for each match

{'filetype': 'image', 'filename': 'TR.jpg', 'created_at': Timestamp('2025-07-21 21:24:38.382369'), 'tokens': '“Speak softly and\ncarry a big stick;\n\nyou will go far.”\n\nTheodore Roosevelt\n', 'summary': '“Speak softly and\ncarry a big stick;\n\nyou will go far.”\n\nTheodore Roosevelt\n', 'entities': ['Theodore Roosevelt'], 'sentiment': {'label': 'POSITIVE', 'score': 0.999}, 'embedding': [0.03546770289540291, -0.0133260702714324, 0.0077492110431194305, -0.014037607237696648, 0.0685034766793251, 0.029060984030365944, 0.01252774428576231, 0.015157049521803856, -0.059621259570121765, -0.05523493513464928, -0.004445046652108431, 0.10148786008358002, -0.06944680213928223, -0.02801528573036194, -0.007897909730672836, 0.04851250350475311, 0.036637771874666214, 0.05237434431910515, -0.017864255234599113, 0.023325705900788307, 0.04691751301288605, 0.03888547420501709, 0.061514146625995636, 0.03131326287984848, -0.04663839191198349, 0.06534938514232635, -0.021461592987179756, -0.07741758227348